In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI管道：评估自定义表格回归模型的批量预测结果

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/custom_tabular_regression_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/custom_tabular_regression_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_evaluation/custom_tabular_regression_model_evaluation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概述

这个笔记本演示了如何使用Vertex AI回归模型评估组件来评估自定义的回归模型。模型评估帮助您根据评估指标确定模型的性能，并在必要时改进模型。

了解更多关于[Vertex AI模型评估](https://cloud.google.com/vertex-ai/docs/evaluation/introduction)和[自定义训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)。

### 目标

在本教程中，您将学习如何使用 `google_cloud_pipeline_components` 通过 Vertex AI 管道作业评估 Vertex AI 模型资源：

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex AI 训练（定制训练）
- Vertex AI 批量预测
- Vertex AI 管道
- Vertex AI 模型注册表

执行的步骤包括：

- 创建一个 Vertex AI `CustomTrainingJob` 用于训练模型。
- 运行 `CustomTrainingJob`
- 检索并加载模型工件。
- 查看模型评估结果。
- 将模型上传为 Vertex AI 模型资源。
- 将一个预训练的 `Vertex AI 模型资源` 导入到管道中。
- 在管道中运行一个 `批量预测` 作业。
- 使用 `回归评估组件` 评估模型。
- 将回归度量指标导入到 Vertex AI 模型资源中。

### 数据集

本教程使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。在本教程中使用的数据集版本已内置在TensorFlow中。训练好的模型可以预测房屋的中位价格，单位为1千美元。

### 费用

本教程使用谷歌云的付费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成费用估算。

## 安装

安装以下必需的软件包以执行此笔记本。

In [ ]:
! pip3 install --upgrade --quiet    google-cloud-aiplatform \
                                    tensorflow \
                                    google-cloud-pipeline-components==1.0.26 \
                                    matplotlib \
                                    google-cloud-storage 

只有Colab可用：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在开始之前

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"
DATA_REGION = "US"

### 认证您的 Google Cloud 账户

根据您的 Jupyter 环境，您可能需要手动进行认证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench**
* 由于您已经通过认证，无需进行任何操作。

**2. 本地 JupyterLab 实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务帐号或其他
* 请参考如何授予您的服务帐号Cloud Storage权限网址：https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

#### 服务账号

您可以使用服务账号来创建 Vertex AI Pipeline 作业。如果您不想使用您项目的 Compute Engine 服务账号，则将 `SERVICE_ACCOUNT` 设置为另一个服务账号ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

设置用于 Vertex AI Pipelines 的服务帐号访问权限

运行以下命令，将您的服务帐号授予读取和写入管道工件的权限，该管道工件位于您在上一步创建的存储桶中。您只需要对每个服务帐号运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 导入库

In [ ]:
import json
import os

import google.cloud.aiplatform as aip
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from google.cloud import aiplatform_v1
from tensorflow.keras.datasets import boston_housing

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和对应的桶初始化用于 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI, location=REGION)

### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`，以使用支持 GPU 的容器映像和分配给虚拟机实例的 GPU 数量。例如，要使用一个 GPU 容器映像，每个 VM 分配 4 个 Nvidia Telsa K80 GPU，您可以指定：

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`以使用一个在 CPU 上运行的容器映像。

了解有关[您所在地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)

*注意*：在 GPU 支持之前 2.3 版本的 TF 将无法加载此教程中的自定义模型。这是一个已知问题，在 TF 2.3 中已修复，原因是在服务函数中生成的静态图操作。如果您在自定义模型上遇到此问题，请使用支持 GPU 的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置预构建的容器

设置用于训练和预测的预构建的Docker容器镜像。


查看最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。


查看最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-9"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个 vCPU 的内存为 3.75GB。
     - `n1-highmem`: 每个 vCPU 的内存为 6.5GB。
     - `n1-highcpu`: 每个 vCPU 的内存为 0.9GB。
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96 \] 中的数字

*注意: 以下内容不支持用于训练：*

 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 和 8 vCPUs

*注意: 您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## 训练自定义模型

现在，您已经准备好开始创建自己的自定义模型并对波士顿房屋进行训练。

了解有关[在Vertex AI上进行自定义模型训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)的更多信息。

### 检查训练包

#### 包布局

在开始训练之前，您可以查看为自定义训练作业组装Python包的方式。解压后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 包含将包安装到Docker镜像的操作环境中的说明。

文件 `trainer/task.py` 是执行自定义训练作业的Python脚本。

**注意：** 当在工作池规范中引用它时，您需要将目录斜杠替换为点号（`trainer.task`），并且去掉文件后缀（`.py`）。

#### 包组装

在下面的单元格中，您将组装训练包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### 创建 task.py

在下一个单元格中，您将编写训练脚本 task.py 的内容。总结如下：

- 从命令行获取保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 获取。
- 从 TF.Keras 内置数据集加载波士顿房屋数据集。
- 使用 TF.Keras 模型 API 构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分布策略。
- 使用指定的 epochs 训练模型（`fit()`）。
- 将训练好的模型保存到指定的模型目录（`save(args.model_dir)`）。
- 将每个特征的最大值保存到指定的参数文件中（`f.write(str(params))`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  
  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  
    
  
  #Get maximum value of each column
  max_value_in_each_column_array_x_train=np.max(x_train,axis=0)

  #dividing each value by the maximum value of that column  
    
  x_train=x_train/max_value_in_each_column_array_x_train

  max_value_in_each_column_array_x_test=np.max(x_test,axis=0)

  #dividing each value by the maximum value of that column  
    
  x_test=x_test/max_value_in_each_column_array_x_test

  params=max_value_in_each_column_array_x_train

  

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

将训练脚本存储在您的云存储桶中。

接下来，您将训练文件夹打包成一个压缩的tar文件，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_boston.tar.gz

### 创建和运行自定义训练作业

要训练一个自定义模型，您需要执行两个步骤：

1) 创建一个自定义训练作业

2) 运行这个作业

#### 创建一个自定义训练作业

使用`CustomTrainingJob`类创建一个自定义训练作业，需要以下参数：

- `display_name`：自定义训练作业的可读名称
- `container_uri`：训练容器镜像
- `requirements`：训练容器镜像的包要求（例如，pandas）
- `script_path`：训练脚本的相对路径

In [ ]:
train_job = aip.CustomTrainingJob(
    display_name="boston",
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(train_job)

#### 准备你的命令行参数

现在为你的自定义训练容器定义命令行参数：

- `args`: 要传递给设置为容器入口点的可执行文件的命令行参数。
  - `--model-dir`: 对于我们的示例，我们使用这个命令行参数来指定存储模型文件的位置。
      - 直接: 你将 Cloud Storage 的位置作为命令行参数传递给训练脚本（设置变量 `DIRECT = True`），或者
      - 间接: 服务将 Cloud Storage 的位置作为环境变量 `AIP_MODEL_DIR` 传递给你的训练脚本（设置变量 `DIRECT = False`）。在这种情况下，你需要在作业规范中告诉服务模型文件的位置。
  - `"--epochs=" + EPOCHS`: 训练的 epoch 数量。
  - `"--steps=" + STEPS`: 每个 epoch 的步骤数量。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, "model")

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

#### 运行自定义训练任务

接下来，通过调用`run()`方法以以下参数运行自定义任务来开始训练工作：

- `args`：要传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作实例的加速器数量。
- `base_output_dir`：将模型工件写入的Cloud Storage位置。
- `sync`：是否同步执行此方法。如果为False，则此方法将在并发Future中执行，并在Future完成时立即返回和同步下游对象。

In [ ]:
if TRAIN_GPU:
    train_job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    train_job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

#### 加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶中加载它，然后您可以执行模型评估和进行预测请求等任务。

要加载模型，您需要将 Cloud Storage 路径 "MODEL_DIR" 传递给 `tf.saved_model.load()` 方法。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

获取服务功能签名

您可以通过重新加载模型并查询相应层的签名来获得模型的输入和输出层签名。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要知道服务功能的输入层名称，以便在进行预测请求时使用。

您还需要知道服务功能的输入和输出层的名称，以便在**稍后的步骤**中构建解释元数据。

In [ ]:
serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

## 配置基于特征的解释（可选）

**要配置模型的解释，按照以下步骤进行。此步骤为可选步骤。**

要使用自定义训练模型的 Vertex 可解释 AI，您必须在创建要请求解释的模型资源时或部署模型时，或提交批量解释作业时配置某些选项。

如果您想要使用 Vertex 可解释 AI 与 AutoML 表格模型，则无需执行任何配置。Vertex AI 会自动为 Vertex 可解释 AI 配置模型。

### 解释规范

在进行预测时获得解释时，您必须在上传自定义模型到 Vertex AI 模型注册表时启用解释功能并设置相应设置。这些设置被称为解释元数据，包括：

- `parameters`：解释您模型的可解释性算法的规范。您可以选择以下之一：
  - Shapley（不建议用于图像数据，因为计算可能会花费很长时间）
  - XRAI
  - 集成梯度
- `metadata`：算法在您的自定义模型上应用的规范

了解更多关于[解释规范](https://cloud.google.com/vertex-ai/docs/explainable-ai/configuring-explanations-feature-based#when-creating-or-importing-model)。

在下一个代码单元格中，您定义参数。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aip.explain.ExplanationParameters(PARAMETERS)

在下一个代码单元格中，您定义元数据

In [ ]:
INPUT_METADATA = {
    "input_tensor_name": serving_input,
    "encoding": "BAG_OF_FEATURES",
    "modality": "numeric",
    "index_feature_mapping": [
        "crim",
        "zn",
        "indus",
        "chas",
        "nox",
        "rm",
        "age",
        "dis",
        "rad",
        "tax",
        "ptratio",
        "b",
        "lstat",
    ],
}

OUTPUT_METADATA = {"output_tensor_name": serving_output}

input_metadata = aip.explain.ExplanationMetadata.InputMetadata(INPUT_METADATA)
output_metadata = aip.explain.ExplanationMetadata.OutputMetadata(OUTPUT_METADATA)

metadata = aip.explain.ExplanationMetadata(
    inputs={"features": input_metadata}, outputs={"medv": output_metadata}
)

### 创建实例模式和预测模式的yaml文件

在接下来的单元格中，您要写instance_schema.yaml和prediction_schema.yaml文件的内容。两个文件的内容结构是相同的。

#### 创建实例模式yaml文件

在下一个单元格中，您要写入instance_schema.yaml的内容。您要写有关您提供给批量预测的预测实例的结构。

- 指定标题和描述。
- 指定输入的类型。在您的情况下，输入层到批量预测的输入是
**{"dense_input": [0.02715405449271202, 0.0, 0.027177177369594574, 0.0, 0.0010195195209234953, 0.009660660289227962, 0.1501501500606537, 0.0027548049110919237, 0.036036036908626556, 1.0, 0.03033033013343811, 0.04091591760516167, 0.043618619441986084]}**
它是一个对象。在对象内部，有像dense_input这样的属性。
- 指定有关属性的描述。
- 对于每个属性，指定其类型。
- 如果属性的类型是数组，请在`items`键中提及有关数组项的信息。

In [ ]:
%%writefile instance_schema.yaml
title: TabularRegression
description: 'Regression Instances.'

type: object
properties:
  dense_input:
    type: array
    items:
      type: float
      minimum: 0.0
      maximum: 1.0
    description: 'Input values to model'


创建预测模式的yaml文件

在下一个单元格中，您将编写prediction_schema.yaml文件的内容。 您要编写有关从批处理预测作业中获得的预测输出的结构。

批处理预测作业的输出为"prediction": [value]，类型为数组。

In [ ]:
%%writefile prediction_schema.yaml
title: TabularRegression
description: 'Regression results.'

type: array

将这两个文件都上传到您的云存储桶中。

In [ ]:
!gsutil cp instance_schema.yaml $BUCKET_URI/instance_schema.yaml
!gsutil cp prediction_schema.yaml $BUCKET_URI/prediction_schema.yaml

### 上传模型

接下来，使用 `Model.upload()` 方法将您的模型上传到一个 `Model` 资源，具有以下参数：

- `display_name`: `Model` 资源的人类可读名称。
- `artifact`: 训练模型 artifacts 的 Cloud Storage 位置。
- `serving_container_image_uri`: 服务容器镜像。
- `instance_schema_uri`: 指向存储在 Google Cloud 存储中描述单个实例格式的 YAML 文件。
- `prediction_schema_uri`: 指向存储在 Google Cloud 存储中描述该模型产生的单个预测格式的 YAML 文件。
- `sync`: 是否异步或同步执行上传。
- `explanation_parameters`: 为 `Model` 的预测配置解释的参数。
- `explanation_metadata`: 描述 `Model` 的输入和输出以解释的元数据。

如果 `upload()` 方法是异步运行的，则可以随后使用 `wait()` 方法阻塞直到完成。

**注意:** 如果要为该模型配置解释，请设置 `explanation_parameters` 和 `explanation_metadata` 参数。否则请不要设置它们。

In [ ]:
model = aip.Model.upload(
    display_name="boston_new_model",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    instance_schema_uri=f"{BUCKET_URI}/instance_schema.yaml",
    prediction_schema_uri=f"{BUCKET_URI}/prediction_schema.yaml",
    explanation_parameters=parameters,
    explanation_metadata=metadata,
    sync=False,
)

model.wait()

### 加载用于流水线的数据

您可以使用`tf.keras.datasets`中的`load_data()`方法加载波士顿房屋测试（保留）数据。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：特征数据和对应的标签（拥有者占用房屋的中位数价值）。

您不需要训练数据，因此您可以加载为`(_, _)`。

在将数据传递到流水线之前，您需要对其进行预处理：

`x_test`：
1. 通过将每个值除以该列的最大值来对每列中的数据进行标准化（重新缩放）。这将用介于0和1之间的32位浮点数替换每个单个值。

In [ ]:
(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)

max_value_in_each_column_array = np.max(x_test, axis=0)


# dividing each value by the maximum value of that column
x_test = x_test / max_value_in_each_column_array


x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])

### 将输入文件添加到管道中

现在创建一个输入文件，并将其存储在本地的云存储桶中。文件中的每个实例都是以下形式的字典条目：

{服务输入：内容，标签列：值}

- `服务输入`：底层模型的输入层的名称。
- `内容`：测试项目的特征值列表。
- `标签列`：为此键提供任何名称。在下面的管道参数中使用相同的名称作为target_field_name。
- `值`：此实例的实际值。

In [ ]:
gcs_input_uri = BUCKET_URI + "/" + "test_file_with_ground_truth.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for i in range(10):
        data = {serving_input: x_test[i].tolist(), "MEDV": y_test[i]}
        f.write(json.dumps(data) + "\n")

## 模型评估

现在，您可以运行 Vertex AI 的 BatchPrediction 作业，并通过使用 `evaluate` 函数创建 Vertex AI 管道，对其结果生成评估和特征归因。了解更多关于 [evaluate 函数](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/models.py#L5127) 的信息。

### 定义参数以运行评估功能

指定运行`evaluate`功能所需的参数。

以下是`evaluate`函数参数的说明：

- `prediction_type`：评估运行处理的问题类型。当前支持的问题类型为'classification'和'regression'。
- `target_field_name`：用作回归目标的列的名称。
- `gcs_source_uris`：批量预测的输入实例的Cloud Storage桶URI列表。
- `generate_feature_attributions`：可选项。模型评估作业是否应生成特征归因。如果未指定，默认值为False。

**管道需要大约1小时才能完成。**

In [ ]:
job = model.evaluate(
    prediction_type="regression",
    target_field_name="MEDV",
    gcs_source_uris=[BUCKET_URI + "/" + "test_file_with_ground_truth.jsonl"],
    generate_feature_attributions=True,
)

print("Waiting model evaluation is in process")
job.wait()

在上一步的结果中，单击生成的链接以在云控制台中查看您的运行。

模型评估流程运行时图

在用户界面中，单击管道DAG节点时，许多节点将展开或折叠。以下是DAG的部分展开视图（单击图像查看更大版本）。

点击了解更多内容<img src="images/custom_tabular_regression_evaluation_pipeline.PNG" style="height:622px;width:726px"></img>

##获取模型评估结果

在评估流程完成后，运行以下单元格以打印评估指标。

In [ ]:
model_evaluation = job.get_model_evaluation()

In [ ]:
# Iterate over the pipeline tasks
for (
    task
) in model_evaluation._backing_pipeline_job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the evaluation task
    if (
        ("model-evaluation" in task.task_name)
        and ("model-evaluation-import" not in task.task_name)
        and (
            task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
            or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
        )
    ):
        evaluation_metrics = task.outputs.get("evaluation_metrics").artifacts[
            0
        ]  # ['artifacts']
        evaluation_metrics_gcs_uri = evaluation_metrics.uri

print(evaluation_metrics)
print(evaluation_metrics_gcs_uri)

### 可视化指标

评估流程完成后，请运行下面的单元格来可视化评估指标。

In [ ]:
metrics = []
values = []
for i in evaluation_metrics.metadata.items():
    # if (
    #     i[0] == "meanAbsolutePercentageError"
    # ):  # we are not considering MAPE as it is infinite. MAPE is infinite if groud truth is 0 as in our case Age is 0 for some instances.
    #     continue
    metrics.append(i[0])
    values.append(i[1])
plt.figure(figsize=(15, 5))
plt.bar(x=metrics, height=values)
plt.title("Evaluation Metrics")
plt.ylabel("Value")
plt.show()

获取特征归因（可选）

**如果您已为模型配置了解释，请运行下面的单元格。否则跳过下面的单元格。**


特征归因表示模型中每个特征对每个给定实例的预测贡献量。

了解更多关于[特征归因](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview#feature_attributions)。

运行下面的单元格以获取特征归因。

In [ ]:
# Iterate over the pipeline tasks
for (
    task
) in model_evaluation._backing_pipeline_job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the feature-attribution task
    if (task.task_name == "feature-attribution") and (
        task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
        or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
    ):
        feat_attrs = task.outputs.get("feature_attributions").artifacts[0]
        feat_attrs_gcs_uri = feat_attrs.uri

print(feat_attrs)
print(feat_attrs_gcs_uri)

从获取的功能归因的云存储URI中获取归因值。

In [ ]:
# Load the results
attributions = !gsutil cat $feat_attrs_gcs_uri

# Print the results obtained
attributions = json.loads(attributions[0])
print(attributions)

### 可视化特征归因

使用柱状图可视化每个特征的获得归因。

In [ ]:
data = attributions["explanation"]["attributions"][0]["featureAttributions"]
features = []
attr_values = []
for key, value in data.items():
    features.append(key)
    attr_values.append(value[0])

plt.figure(figsize=(5, 3))
plt.bar(x=features, height=attr_values)
plt.title("Feature Attributions")
plt.xticks(rotation=90)
plt.ylabel("Attribution value")
plt.show()

清理

要清理此项目中使用的所有谷歌云资源，您可以[删除用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

将`delete_bucket`设置为**True**以删除本笔记本中创建的Cloud Storage存储桶。

In [ ]:
import os

# Delete model resource
model.delete()

# Delete the training job
train_job.delete()

# Delete the evaluation pipeline
job.delete()

# Delete Cloud Storage objects
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI